In [1]:
import pandas as pd
import numpy as np
import os
import urllib.request
import glob
import re
import calendar
import datetime

In [2]:
taxi_Zone_lookup = pd.read_csv("taxi_zone_lookup.csv")
manhattan_taxi_zones = taxi_Zone_lookup[taxi_Zone_lookup["Borough"]=="Manhattan"]

In [3]:
file_list = glob.glob("data/taxis/*")
file_list = pd.DataFrame(file_list).rename(columns={0:"file_name"})
file_list["date"] = file_list["file_name"].str.extract(r'(\d{4}-\d{2})')
file_list["date"] = pd.to_datetime(file_list["date"])
file_list["final_day"] = file_list["date"].apply(lambda x: calendar.monthrange(x.year, x.month)[1])
file_list["end_date"] = file_list.apply(lambda x: x["date"]+datetime.timedelta(days=x["final_day"], hours=23), axis=1)

In [4]:
dfs = {}
for index, row in file_list.iterrows():
    file_df = pd.read_parquet(row["file_name"])
    df = file_df[file_df["PULocationID"].isin(manhattan_taxi_zones["LocationID"])]
    df["tpep_pickup_hour"] = df["tpep_pickup_datetime"].apply(lambda x: x.replace(minute=0, second=0))
    df = df.loc[(df["tpep_pickup_hour"]>=row["date"]) & (df["tpep_pickup_hour"]<=row["end_date"])]
    df_grouped = pd.DataFrame(df.groupby(["tpep_pickup_hour","PULocationID"])["VendorID"].count()).reset_index().rename(columns={"VendorID":"Total_Trips"})
    dfs[row["file_name"]]=df_grouped
    

C:\Users\Acer\AppData\Local\Temp\ipykernel_8236\2169594755.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tpep_pickup_hour"] = df["tpep_pickup_datetime"].apply(lambda x: x.replace(minute=0, second=0))
C:\Users\Acer\AppData\Local\Temp\ipykernel_8236\2169594755.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tpep_pickup_hour"] = df["tpep_pickup_datetime"].apply(lambda x: x.replace(minute=0, second=0))
C:\Users\Acer\AppData\Local\Temp\ipykernel_8236\2169594755.py:5: SettingWithCopyWarning: 
A 

In [11]:
df_final = pd.concat(dfs.values(), ignore_index=True)

In [17]:
df_final.to_parquet("data/merged_dataset.parquet")